<a href="https://colab.research.google.com/github/smygmc/flight-schedule-optimization-with-genetic-algorithm/blob/main/flight_schedule_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import random
import math
import sys

In [2]:
people=[('Lisbon', 'LIS'),
          ('Madrid', 'MAD'),
          ('Paris', 'CDG'),
          ('Dublin', 'DUB'),
          ('Brussels', 'BRU'),
          ('London', 'LHR')] # cities and abbreviations



In [3]:
destination='FCO' # roma airport

In [4]:
flights={}
for row in open("/content/flights.txt"):
  origin,destination,departure_time,arrival_time,cost=row.split(",")
  flights.setdefault((origin,destination),[]) # add key with setdefault so it wont throw an error in first time
  '''
  The setdefault() method returns the value of the item with the specified key. 
  If the key does not exist, insert the key, with the specified value, see example below
  If the key exist, this parameter has no effect.
  If the key does not exist, this value becomes the key's value
  Default value None

  '''
  flights[(origin,destination)].append((departure_time,arrival_time,int(cost)))


In [5]:
print(flights)

{('FCO', 'LIS'): [('6:19', '8:13', 239), ('8:04', '10:59', 136), ('9:31', '11:43', 210), ('11:07', '13:24', 171), ('12:31', '14:02', 234), ('14:05', '15:47', 226), ('15:07', '17:21', 129), ('16:35', '18:56', 144), ('18:25', '20:34', 205), ('20:05', '21:44', 172)], ('LIS', 'FCO'): [('6:11', '8:31', 249), ('7:39', '10:24', 219), ('9:15', '12:03', 99), ('11:08', '13:07', 175), ('12:18', '14:56', 172), ('13:37', '15:08', 250), ('15:03', '16:42', 135), ('16:51', '19:09', 147), ('18:12', '20:17', 242), ('20:05', '22:06', 261)], ('FCO', 'MAD'): [('6:03', '8:43', 219), ('7:50', '10:08', 164), ('9:11', '10:42', 172), ('10:33', '13:11', 132), ('12:08', '14:47', 231), ('14:19', '17:09', 190), ('15:04', '17:23', 189), ('17:06', '20:00', 95), ('18:33', '20:22', 143), ('19:32', '21:25', 160)], ('MAD', 'FCO'): [('6:05', '8:32', 174), ('8:25', '10:34', 157), ('9:42', '11:32', 169), ('11:01', '12:39', 260), ('12:44', '14:17', 134), ('14:22', '16:32', 126), ('15:58', '18:40', 173), ('16:43', '19:00', 24

In [6]:
def print_schedule(schedule):
  flight_id = -1
  total_price = 0
  for i in range(len(schedule) // 2):#divide by 2 because schedule list's length is (number of people*2) for arriving and departure 
    name = people[i][0]
    origin = people[i][1]
    flight_id += 1
    to_rome = flights[(origin, destination)][schedule[flight_id]]
    total_price += to_rome[2]
    flight_id += 1
    from_rome = flights[(destination, origin)][schedule[flight_id]]
    total_price += from_rome[2]
    print(f'{name}, {origin}, {to_rome[0]}, {to_rome[1]}, {to_rome[2]},{from_rome[0]}, {from_rome[1]}, {from_rome[2]}')
  print('Total price: ', total_price)


In [7]:
print_schedule([7,3, 3,2, 7,3, 6,3, 2,4, 5,9])

Lisbon, LIS, 16:51, 19:09, 147,11:07, 13:24, 171
Madrid, MAD, 11:01, 12:39, 260,9:11, 10:42, 172
Paris, CDG, 17:07, 20:04, 291,11:08, 14:38, 262
Dublin, DUB, 15:27, 17:18, 151,10:33, 12:03, 74
Brussels, BRU, 9:08, 12:12, 364,12:20, 16:34, 500
London, LHR, 13:40, 15:38, 137,19:46, 21:45, 214
Total price:  2743


In [8]:
def get_minutes(hour):
  t = time.strptime(hour, '%H:%M')
  minutes = t[3] * 60 + t[4]
  return minutes

In [9]:
get_minutes("3:00")

180

In [26]:
def fitness_function(schedule):
  total_price = 0
  last_arrival_to_rome = 0 #dk cinsinden en fazla (bekleme süresi en az =0)
  first_departure_from_rome = 1439 # dk cinsinden en az (bekleme süresi en az =0)(24  saat sınırları içereisinde bu zaman aralığı 00:00-23:59)

  flight_id = -1
  for i in range(len(schedule) // 2):
    origin = people[i][1]
    flight_id += 1
    to_rome = flights[(origin, destination)][schedule[flight_id]]
    flight_id += 1
    from_rome = flights[(destination, origin)][schedule[flight_id]]

    total_price += to_rome[2]
    total_price += from_rome[2]

    if last_arrival_to_rome < get_minutes(to_rome[1]):#şu anki last arrival dk sından daha fazla olan bir değer se last arrival odur
      last_arrival_to_rome = get_minutes(to_rome[1])
    if first_departure_from_rome > get_minutes(from_rome[0]):#şuanki first departure dk sından daha az bir değerse first departure odur
      first_departure_from_rome = get_minutes(from_rome[0])


  total_wait = 0
  flight_id = -1
  for i in range(len(schedule) // 2):
    origin = people[i][1]
    flight_id += 1
    to_rome = flights[(origin, destination)][schedule[flight_id]]
    flight_id += 1
    from_rome = flights[(destination, origin)][schedule[flight_id]]

    total_wait += last_arrival_to_rome - get_minutes(to_rome[1])
    total_wait += get_minutes(from_rome[0]) - first_departure_from_rome
  return total_wait + total_price

In [11]:
domain=[(0, 9)] * (len(people) * 2)
domain

[(0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9)]

In [12]:
def mutation(domain, schedule, probability):# mutasyon olsun mu olmasın mı belirleyen olasılık değeri
  gene = random.randint(0, len(domain) - 1)# both limit values are included hangi genin mutasyona uğrayacağını belirliyoruz
  mutant = schedule # schedule ı bu listeye atıyoruz (pass by assignment)
  if random.random() < probability: # 0.02, 0.05 
    mutant[gene]=random.randint(domain[gene][0],domain[gene][1])
    '''if schedule[gene] != domain[gene][0]:# 0 a eşitse zaten 1 eksiltemessin
      mutant = schedule[0:gene] + [schedule[gene] - 1] + schedule[gene + 1:] #[0:n] n dahil değil
    else:
      #if schedule[gene] != domain[gene][1]:#9 a eşitse 1 arttıramassın
        mutant = schedule[0:gene] + [schedule[gene] + 1] + schedule[gene + 1:]
        '''
     
  return mutant

In [13]:
def crossover(domain, individual1, individual2):
  gene = random.randint(1, len(domain) - 2)
  return individual1[0:gene] + individual2[gene:]

In [14]:
def genetic_algorithm(domain, fitness_function, population_size = 100, elitism = 0.2,
                      number_of_generations = 200, mutation_probability = 0.05):
  population = []
  for i in range(population_size):
    individual = [random.randint(domain[i][0], domain[i][1]) for i in range(len(domain))]#individual dediği bir tane schedule listesi
    population.append(individual)
  number_elitism = int(elitism * population_size)
  for i in range(number_of_generations):
    costs = [(fitness_function(individual), individual) for individual in population]#popülasyondaki her birey(her bir schedule listesi) için fitness değeri ve kendisini tutan bir tuple listesi
    costs.sort()#küçükten büyüğe sıralıyor
    ordered_individuals = [individual for (cost, individual) in costs]#coststan sadece individualları al
    population = ordered_individuals[0:number_elitism]#0 dan number_elitism e kadar yani en iyi yüzdelik dilime kadar olan kısmı al
    while len(population) < population_size:#tekrar population size kadar doldurucak listeyi nir nesil geçmiş olucak
      i1 = random.randint(0, number_elitism)
      i2 = random.randint(0, number_elitism)
      new_individual = crossover(domain, ordered_individuals[i1], ordered_individuals[i2])
      new_individual_mutation = mutation(domain, new_individual, mutation_probability)
      population.append(new_individual_mutation)
  

  return costs[0][1]
     

In [32]:
x=genetic_algorithm(domain,fitness_function)


In [33]:
fitness_function(x)

2806

In [23]:
print_schedule(x)

Lisbon, LIS, 20:05, 22:06, 261,11:07, 13:24, 171
Madrid, MAD, 19:50, 22:24, 269,18:33, 20:22, 143
Paris, CDG, 19:53, 22:21, 173,12:37, 15:05, 170
Dublin, DUB, 13:40, 15:37, 138,10:33, 12:03, 74
Brussels, BRU, 6:12, 10:22, 230,10:51, 14:16, 256
London, LHR, 18:35, 20:28, 204,10:32, 13:16, 139
Total price:  2228
